In [1]:
from xbbg import blp
import pdblp
import workdays
import datetime
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 70)

In [2]:
T = ["SPXT Index","ES1 Index","SBUS13L Index","SBUS13YC Index","JPY BGN Curncy"]
d_from = "20070301"
d_to = workdays.workday(datetime.datetime.today(), days=-1).strftime("%Y%m%d")
BDH = blp.bdh(T, "px_last", d_from, d_to, Calendar="5D", Fill="P" ).reset_index()

BDH = BDH[["index"]+T]
df = pd.DataFrame(BDH["index"].values,columns = ["Date"])

def ME(df):
    df["ME"] = 0
    for i in range(1,len(df)):
        if df.loc[i,"Date"].month != df.loc[i-1,"Date"].month:
            if df.loc[i-1,"Date"].month ==3:
                df.loc[i-1,"ME"] = 3
            else:
                df.loc[i-1,"ME"] = 1
    df.loc[len(df)-1,"ME"] = 3    
    return df

df = ME(df)


df["SPXT"] = BDH["SPXT Index"].pct_change()
df["JPY"] = BDH["JPY BGN Curncy"].pct_change()
df["SPXT_JPY"] = (BDH["SPXT Index"] * BDH["JPY BGN Curncy"]).pct_change()

df["ES"] = BDH["ES1 Index"].pct_change()
df["HC"] = BDH["SBUS13YC Index"].pct_change() - BDH["SBUS13L Index"].pct_change()

df["ES+JPY"] = df["ES"] - df["HC"] + df["JPY"] 


tmp = ["SPXT_JPY","ES+JPY","HC"]
tmp1 = [ x + "_Index" for x in tmp]
df[tmp1] = (1 + df[tmp]).cumprod().fillna(1) * 100

df

,Date,ME,SPXT,JPY,SPXT_JPY,ES,HC,ES+JPY,SPXT_JPY_Index,ES+JPY_Index,HC_Index
0,2007-03-01,0,NaN,NaN,NaN,NaN,NaN,NaN,100.000000,100.000000,100.000000
1,2007-03-02,0,-0.011400,-0.006972,-0.018292,-0.013708,-0.000140,-0.020539,98.170784,97.946074,99.985959
2,2007-03-05,0,-0.009364,-0.010873,-0.020136,-0.009739,-0.000393,-0.020219,96.194061,95.965720,99.946619
3,2007-03-06,0,0.015536,0.009262,0.024942,0.016942,-0.000114,0.026318,98.593323,98.491309,99.935183
4,2007-03-07,0,-0.002182,-0.004631,-0.006803,-0.001608,-0.000135,-0.006105,97.922561,97.890059,99.921729
...,...,...,...,...,...,...,...,...,...,...,...
4332,2023-10-09,0,0.006304,-0.005425,0.000845,0.006277,-0.000475,0.001327,544.190228,480.699036,77.278324
4333,2023-10-10,0,0.005228,0.001347,0.006582,0.005207,-0.000164,0.006718,547.771975,483.928305,77.265672
4334,2023-10-11,0,0.004341,0.003093,0.007448,0.004156,-0.000157,0.007406,551.851881,487.512406,77.253523
4335,2023-10-12,0,-0.006139,0.004290,-0.001875,-0.006633,-0.000163,-0.002180,550.817277,486.449572,77.240969
